## Notebook created to explore and cluster the neighborhoods in Toronto.

Importing necessary libraries

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Libraries imported.


# Question 1: Scrap content from wikipedia page using Panda Dataframes

About the Data, Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

1.It is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario.
2.Scraping table from HTML using Python Dataframe.

In [4]:
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.rename(columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# Printing the number of rows of the original dataframe
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 3 columns):
PostalCode      287 non-null object
Borough         287 non-null object
Neighborhood    287 non-null object
dtypes: object(3)
memory usage: 6.8+ KB


(287, 3)

# Creating cleaned dataframe

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [6]:
# Printing the columns of original dataframe
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [7]:
# Droping the cells with a borough that is Not assigned
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [8]:
# The dataframe can be reindexed 
df1 = df.reset_index()
df1.head()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M6A,North York,Lawrence Heights
4,6,M6A,North York,Lawrence Manor


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 4 columns):
index           210 non-null int64
PostalCode      210 non-null object
Borough         210 non-null object
Neighborhood    210 non-null object
dtypes: int64(1), object(3)
memory usage: 6.6+ KB


In [10]:
# Printing the number of rows
df1.shape

(210, 4)

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [11]:
# Using GroupBy to combine into one row with two neighborhoods
df2=df1.groupby("PostalCode").agg(lambda x:','.join(set(x)))

In [12]:
df2.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
M1E,Scarborough,"Guildwood,West Hill,Morningside"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [13]:
# Assigning the neighborhood same as the borough if a cell has a borough but a Not assigned neighborhood
df2.loc[df2['Neighborhood']=="Not assigned",'Neighborhood']=df2.loc[df2['Neighborhood']=="Not assigned",'Borough']

In [14]:
# Reseting the index
df3 = df2.reset_index()

In [15]:
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
2,M1E,Scarborough,"Guildwood,West Hill,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
# Print the number of rows in the cleaned dataframe
df3.shape

(103, 3)

# Question 2: Getting the latitude and the longitude coordinates of each neighborhood.

We are using the given csv file to get the geohraphical coordinates of the neighborhoods

In [17]:
df_geocordinate = pd.read_csv("./Geospatial_Coordinates.csv")
df_geocordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
# Combining dataframes "df3" and "df_geocordinate" into one dataframe.
df_toronto = pd.merge(df, df_geocordinate, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


## Question 3: Exploring and clustering the neighborhoods in Toronto.

Getting the latitude and longitude values of Toronto.

In [19]:
# Using ggeopy Library to get the lattitude and longitude values of toronto

address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.653963, -79.387207.


Creating a map of the whole Toronto City with neighborhoods superimposed on top
and adding markers to the map.

In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=9)
map_toronto

# adding markers to the map
for lat, lng, borough, Neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

#### Looking only for the boroughs that contains the word "Toronto".

In [21]:
df_toronto_borough = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_borough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [25]:
# Plotting again the map and adding markers on the map
map_toronto_borough = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_borough['Latitude'], 
        df_toronto_borough['Longitude'], 
        df_toronto_borough['Borough'], 
        df_toronto_borough['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_borough)  

map_toronto_borough

### Using Foursquare API to explore neighborhoods near the  boroughs that contains the word "Toronto", and exploring the trending venues around a location.


#### Define Foursquare Credentials and Version

## A. Exploring the first neighborhood near the boroughs that contains the word "Toronto".

In [36]:
neighborhood_name = df_toronto_borough.loc[0, 'Neighborhood']
print("The first neighbourhood's name is:", neighborhood_name)

The first neighbourhood's name is: Harbourfront


In [37]:
# neighborhood latitude and longitude values
neighborhood_latitude = df_toronto_borough.loc[0, 'Latitude'] 
neighborhood_longitude = df_toronto_borough.loc[0, 'Longitude'] 
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


#### Getting the top 100 venues that are in Harbourfront within a radius of 500 meters.

In [38]:
radius = 500 
LIMIT = 100 # limit of number of venues
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dfbc871dd0f85001bc6e3bb'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'beenHere': {'count': 0,
        'lastCheckinExpiredAt': 0,
        'marked': False,
        'unconfirmedCount': 0},
       'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'contact': {},
       'hereNow': {'count': 0, 'groups': [], 'summary': 'Nobody here'},
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
   

In [39]:
# Extracting the category of the venue by writing Function
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
# Cleaning JSON AND storing it as a pandas dataframe
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Impact Kitchen,Restaurant,43.656369,-79.356980


Number of venues returned by Foursquare

In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


## B. Exploring other neighborhoods in a boroughs that conatin the word Toronto.

We are working on the data frame df_toronto_borough. This region contain Downtown Toronto, East Toronto, North Toronto, Central Toronto.

Firstly, create a function to repeat the same process to all the neighborhoods of Toronto.

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
toronto_borough_venues = getNearbyVenues(names=df_toronto_borough['Neighborhood'],
                                   latitudes=df_toronto_borough['Latitude'],
                                   longitudes=df_toronto_borough['Longitude']
                                  )


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [49]:
toronto_borough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Number of venues returned for each neighborhood.

In [50]:

toronto_borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,14,14,14,14,14,14
Berczy Park,56,56,56,56,56,56
Brockton,23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
CN Tower,14,14,14,14,14,14
Cabbagetown,43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
Chinatown,96,96,96,96,96,96


Let's see number of unique categories from all the returned venues

In [51]:
print('There are {} uniques categories.'.format(len(toronto_borough_venues['Venue Category'].unique())))

There are 236 uniques categories.


## C. Analyzing Each Neighborhood

In [53]:
# one hot encoding
toronto_borough_onehot = pd.get_dummies(toronto_borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_borough_onehot['Neighborhood'] = toronto_borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_borough_onehot.columns[-1]] + list(toronto_borough_onehot.columns[:-1])
toronto_borough_onehot = toronto_borough_onehot[fixed_columns]

toronto_borough_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category



In [54]:
toronto_borough_grouped = toronto_borough_onehot.groupby('Neighborhood').mean().reset_index()
toronto_borough_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Adelaide,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,...,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0
1,Bathurst Quay,0.0,0.0,0.071429,0.071429,0.142857,0.142857,0.142857,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
2,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.00,0.0
3,Brockton,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0


#### Check the 10 most common venues in each neighborhood.

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_borough_grouped['Neighborhood']

for ind in np.arange(toronto_borough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_borough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Thai Restaurant,Steakhouse,Burger Joint,Sushi Restaurant,Asian Restaurant,Bakery,Salad Place,Bar
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Plane,Coffee Shop,Boutique,Airport Food Court
2,Berczy Park,Coffee Shop,Café,Seafood Restaurant,Bakery,Cocktail Bar,Steakhouse,Beer Bar,Cheese Shop,Farmers Market,Basketball Stadium
3,Brockton,Performing Arts Venue,Bakery,Café,Breakfast Spot,Coffee Shop,Stadium,Intersection,Italian Restaurant,Restaurant,Furniture / Home Store
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Smoke Shop,Garden Center,Burrito Place,Auto Workshop,Spa,Fast Food Restaurant,Farmers Market,Garden,Pizza Place


# D. Cluster neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [60]:
kclusters = 5

toronto_borough_grouped_clustering = toronto_borough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_borough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 0, 0, 4, 0, 0, 0, 0])

In [65]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

toronto_borough_merged = df_toronto_borough

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_borough_merged = toronto_borough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_borough_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,0.0,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Ice Cream Shop
1,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0.0,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Lingerie Store,Italian Restaurant,Bubble Tea Shop,Pizza Place
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0.0,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Lingerie Store,Italian Restaurant,Bubble Tea Shop,Pizza Place
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,0.0,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Clothing Store,Diner,Park,Breakfast Spot,Hotel


# E. Examining Clusters

Now, examining each cluster and determining the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [69]:
toronto_borough_merged.loc[toronto_borough_merged['Cluster Labels'] == 0, toronto_borough_merged.columns[[1] + list(range(5, toronto_borough_merged.shape[1]))]]

,Borough,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,0.0,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Ice Cream Shop
2,Downtown Toronto,0.0,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Lingerie Store,Italian Restaurant,Bubble Tea Shop,Pizza Place
3,Downtown Toronto,0.0,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Lingerie Store,Italian Restaurant,Bubble Tea Shop,Pizza Place
4,Downtown Toronto,0.0,0.0,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Clothing Store,Diner,Park,Breakfast Spot,Hotel
5,East Toronto,0.0,0.0,Health Food Store,Trail,Pub,Wings Joint,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant
6,Downtown Toronto,0.0,0.0,Coffee Shop,Café,Seafood Restaurant,Bakery,Cocktail Bar,Steakhouse,Beer Bar,Cheese Shop,Farmers Market,Basketball Stadium
7,Downtown Toronto,0.0,0.0,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Sandwich Place,Burger Joint,Juice Bar,Spa,Bar,Japanese Restaurant
8,Downtown Toronto,0.0,0.0,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Restaurant,Baby Store,Candy Store,Convenience Store
9,Downtown Toronto,0.0,0.0,Coffee Shop,Café,Thai Restaurant,Steakhouse,Burger Joint,Sushi Restaurant,Asian Restaurant,Bakery,Salad Place,Bar
10,Downtown Toronto,0.0,0.0,Coffee Shop,Café,Thai Restaurant,Steakhouse,Burger Joint,Sushi Restaurant,Asian Restaurant,Bakery,Salad Place,Bar


#### Cluster 2

In [70]:
toronto_borough_merged.loc[toronto_borough_merged['Cluster Labels'] == 1, toronto_borough_merged.columns[[1] + list(range(5, toronto_borough_merged.shape[1]))]]

,Borough,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Central Toronto,1.0,1.0,Garden,Wings Joint,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 3

In [72]:
toronto_borough_merged.loc[toronto_borough_merged['Cluster Labels'] == 2, toronto_borough_merged.columns[[1] + list(range(5, toronto_borough_merged.shape[1]))]]

,Borough,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Central Toronto,2.0,2.0,Park,Bus Line,Swim School,Wings Joint,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
34,Central Toronto,2.0,2.0,Park,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
35,Central Toronto,2.0,2.0,Park,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
66,Downtown Toronto,2.0,2.0,Park,Playground,Trail,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 4

In [74]:
toronto_borough_merged.loc[toronto_borough_merged['Cluster Labels'] == 3, toronto_borough_merged.columns[[1] + list(range(5, toronto_borough_merged.shape[1]))]]

,Borough,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Central Toronto,3.0,3.0,Gym,Playground,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
50,Central Toronto,3.0,3.0,Gym,Playground,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 5

In [75]:
toronto_borough_merged.loc[toronto_borough_merged['Cluster Labels'] == 4, toronto_borough_merged.columns[[1] + list(range(5, toronto_borough_merged.shape[1]))]]

,Borough,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,4.0,4.0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Plane,Coffee Shop,Boutique,Airport Food Court
60,Downtown Toronto,4.0,4.0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Plane,Coffee Shop,Boutique,Airport Food Court
61,Downtown Toronto,4.0,4.0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Plane,Coffee Shop,Boutique,Airport Food Court
62,Downtown Toronto,4.0,4.0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Plane,Coffee Shop,Boutique,Airport Food Court
63,Downtown Toronto,4.0,4.0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Plane,Coffee Shop,Boutique,Airport Food Court
64,Downtown Toronto,4.0,4.0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Plane,Coffee Shop,Boutique,Airport Food Court
65,Downtown Toronto,4.0,4.0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Plane,Coffee Shop,Boutique,Airport Food Court
